In [1]:
from proxy_methods_final import *
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_rel
from scipy.stats import wilcoxon
import scipy.stats as stats
from scipy.stats import binom_test
from proxy_variance import *

### Import final dataframe

In [2]:
proxy_time_series_final = pd.read_csv('data/proxy_time_series_final_new.csv')
proxy_time_series_final

,Ticker,Date,Actual_Spread,Proxy_Intersection,Proxy_Intersection_Community,Proxy_CSRA,Proxy_CSRA_Community
0,ABE,2010-09-14,252.500000,191.274580,191.274580,155.297648,165.626455
1,ABE,2010-09-15,253.553505,183.972671,183.972671,154.766746,163.639526
2,ABE,2010-09-16,253.478186,180.312309,180.312309,154.467431,163.054781
3,ABE,2010-09-17,253.417889,178.475907,178.475907,153.254600,162.743968
4,ABE,2010-09-20,250.460282,177.564172,177.564172,153.333533,161.973104
...,...,...,...,...,...,...,...
674015,ZINCO,2020-04-28,42.449000,58.674400,58.674400,58.237550,73.837823
674016,ZINCO,2020-04-29,40.402900,59.070400,59.070400,58.209534,73.485923
674017,ZINCO,2020-04-30,39.661100,56.307100,56.307100,56.308142,70.943173
674018,ZINCO,2020-05-01,42.251900,58.610300,58.610300,57.569103,73.292172


## aggregate the data

In [3]:
proxy_time_series_final_aggregated = proxy_time_series_final.groupby('Date').mean().reset_index()
 
proxy_time_series_final_aggregated

C:\Users\dsvin\AppData\Local\Temp\ipykernel_13660\425146724.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  proxy_time_series_final_aggregated = proxy_time_series_final.groupby('Date').mean().reset_index()


,Date,Actual_Spread,Proxy_Intersection,Proxy_Intersection_Community,Proxy_CSRA,Proxy_CSRA_Community
0,2010-09-14,139.720914,135.125631,133.627149,116.517708,118.078909
1,2010-09-15,140.174020,135.539837,134.023218,117.067594,118.633729
2,2010-09-16,139.795124,135.046863,133.686297,116.592023,118.156712
3,2010-09-17,138.890629,134.322076,133.058153,116.073894,117.634980
4,2010-09-20,139.432729,135.027898,134.016580,116.611033,118.150323
...,...,...,...,...,...,...
2510,2020-04-28,112.804007,112.136381,111.320039,85.941858,89.037764
2511,2020-04-29,112.514192,111.510316,110.447164,85.664666,88.669752
2512,2020-04-30,111.208099,110.463996,109.671745,84.860412,87.924516
2513,2020-05-01,112.549156,111.977504,111.354093,85.775404,88.749648


In [4]:
intersection_variance, intersection_community_variance, csra_variance, csra_community_variance = calculate_variance_aggregated(proxy_time_series_final_aggregated)

In [5]:
print(intersection_variance, intersection_community_variance, csra_variance, csra_community_variance)

0.00018556835537542572 0.0002453253715653261 0.00034755863366821657 0.00021075806138923258


#### mean of variances

In [7]:
metadata = pd.read_csv('data/new_metadata_after_aggregation.csv')

variance_df = pd.DataFrame(columns=['Ticker', 'Intersection Variance', 'Intersection Community Variance', 'CSRA Variance', 'CSRA Community Variance'])

for ticker in metadata['Ticker']:
    
    intersection_variance, intersection_community_variance, csra_variance, csra_community_variance = calculate_variance(ticker, proxy_time_series_final)
    
    new_row = pd.DataFrame([{'Ticker': ticker, 'Intersection Variance': intersection_variance, 'Intersection Community Variance': intersection_community_variance, 'CSRA Variance': csra_variance, 'CSRA Community Variance': csra_community_variance}])

    variance_df = pd.concat([variance_df, new_row], ignore_index=True)

variance_df.to_csv('data/proxy_variance.csv', index=False)

In [8]:
variance_df

,Ticker,Intersection Variance,Intersection Community Variance,CSRA Variance,CSRA Community Variance
0,ABE,0.351242,0.351242,0.022904,0.045117
1,ABHLTD,0.055204,0.062833,0.032204,0.039035
2,ACAFP,0.080598,0.036534,0.077444,0.026301
3,ACAFP-CIB,0.236055,0.459240,0.127883,0.310503
4,ACCOR,0.022002,0.016029,0.023782,0.022555
...,...,...,...,...,...
263,WENL,0.031540,0.029913,0.011920,0.023952
264,WOLKLU,0.261473,0.204732,0.178116,0.089476
265,WPPGRP-2005,0.320779,0.165204,0.108855,0.085554
266,YORPOW,0.096509,0.119917,0.094827,0.085026


In [9]:
vaiance_df_means = variance_df.mean()
vaiance_df_means

C:\Users\dsvin\AppData\Local\Temp\ipykernel_13660\1248991758.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  vaiance_df_means = variance_df.mean()


Intersection Variance              0.311028
Intersection Community Variance    0.251745
CSRA Variance                      0.181462
CSRA Community Variance            0.151663
dtype: float64

In [10]:
metadata

,Ticker,ShortName,Sector,Region,Country,AverageRating
0,ABE,Abertis Infraestructuras SA,Cyclical,Europe,Spain,medium
1,ABHLTD,Alliance Boots Hldgs Ltd,Cyclical,Europe,United Kingdom,medium
2,ACAFP,Cr Agricole SA,Financials,Europe,France,low
3,ACAFP-CIB,Cr Agricole Corporate and Invt Bk,Financials,Europe,France,medium
4,ACCOR,ACCOR,Cyclical,Europe,France,medium
...,...,...,...,...,...,...
263,WENL,WENDEL,Financials,Europe,France,medium
264,WOLKLU,Wolters Kluwer N V,Cyclical,Europe,Netherlands,medium
265,WPPGRP-2005,WPP 2005 Ltd,Cyclical,Europe,United Kingdom,high
266,YORPOW,Yorkshire Pwr Gp Ltd,Non Cyclical,Europe,United Kingdom,medium


In [11]:
merged_df = pd.merge(variance_df, metadata, on="Ticker")

# Calculate the mean variance grouped by Sector
sector_mean_variance = merged_df.groupby('Sector')[[
    'Intersection Variance', 
    'Intersection Community Variance', 
    'CSRA Variance', 
    'CSRA Community Variance'
]].mean().reset_index()

# Calculate the mean variance grouped by AverageRating
rating_mean_variance = merged_df.groupby('AverageRating')[[
    'Intersection Variance', 
    'Intersection Community Variance', 
    'CSRA Variance', 
    'CSRA Community Variance'
]].mean().reset_index()

# Calculate the mean variance grouped by Country
country_mean_variance = merged_df.groupby('Country')[[
    'Intersection Variance', 
    'Intersection Community Variance', 
    'CSRA Variance', 
    'CSRA Community Variance'
]].mean().reset_index()

# Display results
print("Mean Variance by Sector:")
print(sector_mean_variance)

print("\nMean Variance by Rating:")
print(rating_mean_variance)

print("\nMean Variance by Country:")
print(country_mean_variance)

Mean Variance by Sector:
         Sector  Intersection Variance  Intersection Community Variance  \
0      Cyclical               0.511755                         0.311210   
1    Financials               0.113239                         0.185812   
2  Non Cyclical               0.201865                         0.227460   

   CSRA Variance  CSRA Community Variance  
0       0.284587                 0.176366  
1       0.089227                 0.114721  
2       0.115086                 0.152054  

Mean Variance by Rating:
  AverageRating  Intersection Variance  Intersection Community Variance  \
0          high               0.230396                         0.278680   
1           low               0.138020                         0.149254   
2        medium               0.396525                         0.289575   

   CSRA Variance  CSRA Community Variance  
0       0.122526                 0.121665  
1       0.267053                 0.167703  
2       0.156123                 0.1501

In [12]:
sector_mean_variance

,Sector,Intersection Variance,Intersection Community Variance,CSRA Variance,CSRA Community Variance
0,Cyclical,0.511755,0.311210,0.284587,0.176366
1,Financials,0.113239,0.185812,0.089227,0.114721
2,Non Cyclical,0.201865,0.227460,0.115086,0.152054


In [13]:
country_mean_variance

,Country,Intersection Variance,Intersection Community Variance,CSRA Variance,CSRA Community Variance
0,Austria,3.914592,0.102766,1.126287,0.400004
1,Belgium,0.427632,0.214810,0.261358,0.286324
2,Denmark,0.129426,0.129426,0.167138,0.203770
3,Finland,0.345749,0.119654,0.225732,0.459831
4,France,0.144772,0.176715,0.094805,0.104930
5,Germany,0.265679,0.335179,0.140201,0.174029
6,Ireland,0.052872,0.052872,0.051941,0.069256
7,Italy,0.214408,0.559336,0.117197,0.217339
8,Luxembourg,0.071150,0.071150,0.198142,0.198142
9,Netherlands,0.270136,0.399195,0.097224,0.110239


In [14]:
rating_mean_variance

,AverageRating,Intersection Variance,Intersection Community Variance,CSRA Variance,CSRA Community Variance
0,high,0.230396,0.278680,0.122526,0.121665
1,low,0.138020,0.149254,0.267053,0.167703
2,medium,0.396525,0.289575,0.156123,0.150167


In [15]:
merged_df

,Ticker,Intersection Variance,Intersection Community Variance,CSRA Variance,CSRA Community Variance,ShortName,Sector,Region,Country,AverageRating
0,ABE,0.351242,0.351242,0.022904,0.045117,Abertis Infraestructuras SA,Cyclical,Europe,Spain,medium
1,ABHLTD,0.055204,0.062833,0.032204,0.039035,Alliance Boots Hldgs Ltd,Cyclical,Europe,United Kingdom,medium
2,ACAFP,0.080598,0.036534,0.077444,0.026301,Cr Agricole SA,Financials,Europe,France,low
3,ACAFP-CIB,0.236055,0.459240,0.127883,0.310503,Cr Agricole Corporate and Invt Bk,Financials,Europe,France,medium
4,ACCOR,0.022002,0.016029,0.023782,0.022555,ACCOR,Cyclical,Europe,France,medium
...,...,...,...,...,...,...,...,...,...,...
263,WENL,0.031540,0.029913,0.011920,0.023952,WENDEL,Financials,Europe,France,medium
264,WOLKLU,0.261473,0.204732,0.178116,0.089476,Wolters Kluwer N V,Cyclical,Europe,Netherlands,medium
265,WPPGRP-2005,0.320779,0.165204,0.108855,0.085554,WPP 2005 Ltd,Cyclical,Europe,United Kingdom,high
266,YORPOW,0.096509,0.119917,0.094827,0.085026,Yorkshire Pwr Gp Ltd,Non Cyclical,Europe,United Kingdom,medium
